# Generative Models by using Markov Models

**Grandfather of Chatgpt**

### Differences between classification and generative models:
* No Matrices, Only a Dictionary: We will be working with sparse data. Using matrices will bloat the memory because most words don't follow each other. We will only keep track of "existing" transitions. 
* No Smoothing: This is very important. When creating classifiers, we said "let's give a chance to the unseen." But when writing poetry, we don't want made-up words. We want the model to only make the actual transitions it sees. 
* No Logarithms: To be able to sample, we need real probabilities between $0$ and $1$. (e.g., 20% probability, 50% probability). Logarithms disrupt this ratio.

In [260]:
import numpy as np
import re

In [261]:
def get_data(TXT_DIR):
    line_list = []
    TXT_DIR = "../"+TXT_DIR
    with open(TXT_DIR, 'r', encoding='utf-8') as file:
        for line in file:
            #normalization
            line = line.strip()
            line = line.lower()
            line = re.sub(r'[^\w\s]', '', line) #substitute (replace)
            line = re.sub(r'\d', '', line)
            if line != "":
                tokens = line.split()
                line_list.append(tokens) #Markov models want this format. 
    return line_list

In [262]:
frost_list = get_data('data/robert_frost.txt')

In [263]:
#Since strings in python are immutable, editing is slow
#Faster way:

#create empty set
word_set = set()
#directly add words into set
for sentence in frost_list:
    word_set.update(sentence)
idx2word = list(word_set)
idx2word.insert(0, '<UNKNOWN>')

word2idx = {word:i for i, word in enumerate(idx2word)}
print(len(word2idx))
print(list(word2idx.items())[0:5])

2192
[('<UNKNOWN>', 0), ('chanced', 1), ('straight', 2), ('maybe', 3), ('half', 4)]


In [264]:
def word_numerizer(arr):
    int_list = []
    for sentence in arr:
        sample = [word2idx.get(word, 0) for word in sentence]
        int_list.append(sample)
    return int_list

In [265]:
frost_list_int = word_numerizer(frost_list)

In [266]:
print(frost_list[10])
print(frost_list_int[10])

['and', 'both', 'that', 'morning', 'equally', 'lay']
[895, 325, 611, 1713, 1685, 1625]


In [267]:
# Prepare A1, A2 and pi

pi = {}
A1 = {}
A2 = {}

for sentence in frost_list_int: 
    #pi
    if sentence[0] not in pi:
        pi[sentence[0]] = 1
    else:
        pi[sentence[0]] +=1
    #A1
    if len(sentence) >=2:
        first_word = sentence[0]
        second_word = sentence[1]
        if first_word not in A1:
            A1[first_word] = {}
            A1[first_word][second_word] = 1
        else:
            if second_word not in A1[first_word]:
                A1[first_word][second_word] = 1
            else:
                A1[first_word][second_word] +=1 
    #A2
    for word_index in range(len(sentence)-2):
        #key
        current_word = sentence[word_index]
        second_word  = sentence[word_index+1]
        third_word   = sentence[word_index+2]

        keytuple = (current_word, second_word)

        if keytuple not in A2:
            A2[keytuple] = {}
            A2[keytuple][third_word] = 1                    
        else:
            if third_word not in A2[keytuple]:
                A2[keytuple][third_word]= 1
            else:
                A2[keytuple][third_word]+= 1

In [268]:
# Matrices as probabilities

# FOR pi
def calc_pi_probs(dict_):
    #1- Not using log probabilities. We need real probabilities
    #2- No smoothing (+1). Model will not use word combinations 
    # that never appeared in training test. 
    new_pi = {}
    total_count = 0
    for val in dict_.values():
        total_count += val

    for key, value in dict_.items():
        new_pi[key] = (value)/(total_count)
    return new_pi

# FOR A
def calc_A_probsprobabilities(dict_):
    new_A = {}
    
    for key in dict_.keys():
        new_pi = calc_pi_probs(dict_[key])
        new_A[key] = new_pi
    
    return new_A

In [269]:
pi_probs = calc_pi_probs(pi)
A1_probs = calc_A_probsprobabilities(A1)
A2_probs = calc_A_probsprobabilities(A2)

In [270]:
pi_probs.keys()

dict_keys([618, 895, 768, 1643, 559, 1237, 581, 1641, 2079, 64, 1992, 704, 1367, 942, 991, 836, 385, 1454, 1476, 1488, 1954, 791, 74, 479, 1036, 882, 1383, 243, 971, 1072, 1492, 147, 4, 2060, 1797, 881, 2038, 1798, 1513, 616, 173, 75, 1313, 2100, 399, 551, 1274, 1431, 1892, 611, 1473, 320, 1259, 1002, 1676, 659, 1210, 1126, 669, 1331, 55, 719, 277, 1737, 760, 1542, 2071, 306, 1382, 1510, 1319, 936, 774, 433, 2004, 174, 2024, 841, 19, 184, 707, 1832, 1158, 2036, 1429, 1439, 68, 126, 1770, 1329, 1043, 2005, 45, 1493, 1495, 1167, 749, 998, 2085, 624, 207, 1778, 970, 537, 291, 48, 2054, 262, 1762, 2094, 228, 11, 1842, 2093, 835, 224, 415, 1704, 1974, 891, 1609, 872, 924, 851, 598, 1764, 1748, 357, 272, 2072, 1800, 1215, 893, 1822, 1272, 52, 1066, 1853, 785, 1662, 1813, 1385, 514, 352, 2162, 1055, 1574, 325, 427, 444, 378, 413, 278, 798, 24, 331, 914, 1717, 1006, 1276, 59, 549, 1784, 222, 1780, 345, 423, 436, 682, 2063, 1695, 1895, 645, 1350, 570, 1556, 990, 1939, 1039, 1355, 2092, 916, 773

In [271]:
#select random first word:

first_word = np.random.choice([*pi_probs.keys()], p=[*pi_probs.values()])
second_word = np.random.choice([*A1_probs[first_word].keys()], p=[*A1_probs[first_word].values()])
search_key = (first_word, second_word)
third_word = np.random.choice([*A2_probs[search_key].keys()], p=[*A2_probs[search_key].values()])
search_key = (second_word, third_word)
fourth_word = np.random.choice([*A2_probs[search_key].keys()], p=[*A2_probs[search_key].values()])

In [272]:
print(first_word)
print(second_word)
print(third_word)
print(fourth_word)

1510
611
1431
970


In [273]:
idx2word = {}
for v, i in word2idx.items():
    idx2word[i] = v

In [274]:
def generate_poem_int(POEM_SIZE = 4):
    POEM = []
    while True:
        poem_tokens = []
        while True:
            first_word = np.random.choice([*pi_probs.keys()], p=[*pi_probs.values()])
            if first_word in A1_probs:
                second_word = np.random.choice([*A1_probs[first_word].keys()], p=[*A1_probs[first_word].values()])
                break
        poem_tokens = [first_word, second_word]
        while len(poem_tokens)<6:
            search_key = (poem_tokens[-2], poem_tokens[-1])
            if search_key in A2_probs:
                next_word = np.random.choice([*A2_probs[search_key].keys()], p=[*A2_probs[search_key].values()])
                poem_tokens.append(next_word)
            else:
                break
        if len(poem_tokens) == 6:
            POEM.append(poem_tokens)
            if len(POEM) == POEM_SIZE:
                break
    return POEM

def generate_poem(POEM_SIZE = 4):
    POEM = generate_poem_int(POEM_SIZE)
    POEM_str = ""
    for line in POEM:
        poem_line = []
        for token in line:
            poem_line.append(idx2word[token])
        POEM_str += ' '.join(poem_line) + '\n'
    return POEM_str

In [278]:
print(generate_poem())
print(generate_poem())
print(generate_poem())
print(generate_poem())

you and i know enough of
i take it as i am
to stand together on the cellar
strung on your hair wont hurt

i wonder if hes a day
i stole the goblet from the
one level higher than the run
let them will we come to

weep for what little things could
son you wouldnt want to know
yes its important though you think
the origin of all her books

or i cant give myself to
and in a wood and i
son but the pipes there and
especially in winter when the wind



In [279]:
print(generate_poem())
print(generate_poem())
print(generate_poem())
print(generate_poem())

a moment he stood beside and
a chimney that only would serve
old davis owned a solid mica
i was young i used to

all fresh and sound from the
as married thats what i came
they bad been brought home from
and kick with two legs like

if you can be or anyone
that everyone for miles could see
two roads diverged in a town
moving a flock of hens from

its as sound as the day
then shook his lantern saying iles
so now theyve dragged it through
up one flight from the sense

